In [10]:
import pandas as pd
import numpy as np
import librosa
import IPython.display as ipd
from librosa.display import specshow
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("data/data HS.csv")
data = data.apply(pd.to_numeric)
data.index = range(len(data.index))
data = data.sort_values(by=['Temps (min)'])
data.loc[:,'2905.mzML':] = data.loc[:,'2905.mzML':].applymap(np.log)

In [3]:
data.describe()

,Masse (mz),Temps (min),2905.mzML,2915.mzML,2910.mzML,2908.mzML,2907.mzML,2909.mzML,2920.mzML,2921.mzML,...,3063.mzML,3057.mzML,3067.mzML,3069.mzML,3070.mzML,3064.mzML,3071.mzML,3072.mzML,3073.mzML,Unnamed: 171
count,953.000000,953.000000,257.000000,597.000000,938.000000,942.000000,941.000000,936.000000,640.000000,621.000000,...,834.000000,849.000000,687.000000,789.000000,754.000000,821.000000,801.000000,813.000000,937.000000,0.0
mean,401.237266,12.775593,11.926353,12.317468,13.998088,13.967346,13.958635,14.013134,12.429794,12.245494,...,13.453933,13.274097,12.055314,12.620999,12.968401,13.423927,13.207887,13.055290,13.844849,NaN
std,182.765170,9.082042,3.511650,2.762897,1.831448,1.897797,1.894486,1.805374,2.672964,2.684910,...,2.422707,2.596308,2.773044,2.570495,2.681348,2.513927,2.661562,2.722833,1.840990,NaN
min,100.076200,0.070000,7.050952,7.369868,7.554560,7.029021,7.407205,6.940615,7.408613,7.105203,...,6.520625,6.636911,6.551126,6.657372,6.438135,6.437472,6.816665,6.403856,6.521257,NaN
25%,251.090900,3.450000,8.777498,10.030649,12.992711,12.971277,12.951902,12.995503,10.459824,10.278623,...,12.109234,11.729512,9.878946,10.856337,11.160825,11.966013,11.401371,11.329269,12.812267,NaN
50%,385.221100,12.790000,11.137672,12.355275,14.028820,14.017943,14.004657,14.035700,12.459602,12.183468,...,13.888548,13.761646,12.198202,12.772904,13.489097,13.837531,13.518024,13.470308,13.887349,NaN
75%,529.268100,17.630000,14.951587,14.374544,15.034141,15.048209,15.018363,15.033756,14.412426,14.161099,...,15.017013,15.130607,14.122943,14.526844,14.888700,15.196967,15.128743,15.071262,14.949742,NaN
max,799.563300,33.730000,21.043190,21.701000,20.658458,21.502158,20.729492,21.541948,21.224922,21.407015,...,20.201698,20.900589,20.706083,21.043648,20.481150,21.070608,21.109835,20.430192,20.961053,NaN


In [12]:
# Structure d'entrée attendue : vecteur taille m et matrice taille n*m avec : 
# n = résolution_temporelle*durée_expérience = nombre de relevés pendant l'expérience
# m = nombre de molécules distinctes ("Masse (mz)" distincte)

def son(mass_list,abundance_matrix,sr=22050,duration=10):
    n = len(abundance_matrix)
    m = len(mass_list)
    durationOfOneSample = duration*sr/n
    
    amplitude_matrix = abundance_matrix / np.max(abundance_matrix) # On ramène l'amplitude dans ]0,1]
    
    sounds = np.array([[0]*m]*(duration*sr)) # Chaque colonne est une molécule différente, chaque ligne est une unité temporelle
    
    for i in range(duration*sr): # Parcours par temps croissant (on suppose la matrice d'abondance triée)
        sampleNumber = int(i/durationOfOneSample) # Padding. On pourrait remplacer ça par une interpolation
        t = i/sr # temps
        
        for j in range(m): # Parcours de toutes les molécules pour un instant donné
            freq = mass_list[j] # fréquence
            amp = abundance_matrix[i][j] # amplitude
            sounds[i][j] = amp * np.sin(2*np.pi*f*t)
    
    return np.mean(sounds,axis=1) # Fusion des channels de toutes les molécules

In [ ]:
# Génération du fichier audio

sr = 22050 # sampling rate (fréquence d'échantillonage)
duration = 30

audio = ### son(mass_list, abundance_matrix, sr, duration) ###
display(ipd.Audio(audio, rate=sr))
plt.plot(np.arange(0,d,1/sr),audio)

In [ ]:
# Affichage du spectrogramme

plt.figure(figsize=(12, 6))

y_stft = librosa.stft(np.array(audio), win_length=1024)
y_spectrogram = librosa.amplitude_to_db(np.abs(y_stft))
specshow(y_spectrogram, cmap="magma", x_axis="time", y_axis="hz")
plt.title("Short-term Fourier transform")
plt.colorbar()
plt.clim(-40, 30)
plt.xlabel("Time (seconds)")
plt.xlim(0, d)
plt.ylabel("Frequency (Hz)")
plt.ylim(0, 1000)